In [ ]:
# Now that we have vOTUs, we can do all sorts of stuff!
# For now, lets take our vOTUs and compare them to previously described bee phage
# To do this, we will use...
    # Prokka to annotate our phage seqs (we need the .faa files this tool generates for the next tools)
    # vContact2 to construct gene sharing networks
    # clinker to make nice multi genome alignment plots

In [ ]:
# Annotate phage with prokka
# Once we have all of our phage, plus previously described phage in one place, we will annotate them using prokka
# This is an example of the script used to do the annotation

#!/bin/bash
#SBATCH --job-name=prokka_%j
#SBATCH --nodes=1
#SBATCH -t 3:00:00
#SBATCH --ntasks=6
#SBATCH --output=/home/dsbard/bee_phage/reviewer_approach/annot/log/prokka%j.out
#SBATCH --partition=med

conda activate prokka
# module load ncbi-blast

prokka --kingdom Bacteria $1  \
--outdir /home/dsbard/bee_phage/reviewer_approach/annot/annotations/${1%%.fa} \
--hmms /home/dsbard/phrogs/all_phrogs.hmm \
--prefix annotate_${1%%.fa}

# we run this from a directory which contains all phage (known and novel) as seperate files
## /home/dsbard/bee_phage/reviewer_approach/drep/dereplicated_phage/dereplicated_genomes
# This script is run with a forloop (like our previous ones)
cd /home/dsbard/bee_phage/reviewer_approach/annot/all_phage
for f in *.fa
do
sbatch ../../scripts/annot.sh $f
done

# The only additional thing to note here is that we are using Phrogs hidden markov model db to annotate
# This is the website: https://phrogs.lmge.uca.fr/
# The paper: https://academic.oup.com/nargab/article/3/3/lqab067/6342220?login=false
# The actual .hmm I used was formated for Prokka use by the Millardlab and can be found here: https://millardlab.org/2021/11/21/phage-annotation-with-phrogs/
# A big thanks to all these people for making this available!!!

# Once we have run the genomes through prokka, we can pull their .faa files
for f in */*.faa
do 
cp $f ../faa_files/
done
# For whatever reason, prokka doesnt always get rid of intermediate files
## remove these 
rm *tmp*
# Additionally, for whatever reason, 4 of the phage identified in busby et al do not get annotated (produce empty files)
## we can find these with 
find . -size 0
# I think this is because prokka is unable to find any proteins in these files
#busby_phage_123.faa
#busby_phage_72.faa
#busby_phage_126.faa
#busby_phage_91.faa
# remove them
find . -size 0 -delete


In [ ]:
# Run vContact2
# We can now take the .faa files produced by prokka and use them to build a gene-sharing network!
# This will be used to compare the "relatedness" of our phage to those previously described by other bee phage papers
# We can also choose to run this with a collection of reference phage provided by vContact.
    # This is a good thing to do, as it will place our phage (and bee phage more generally) within the context of what we know about phage more generally

##### Important #####
# Before we can run this script, we need to prepare the all_known_gene2genome.tsv file
# This is a file which contains 3 columns 
    # 1) Protein ID (a unique identified for each protein)
    # 2) Contig ID (Phage name -- an indentified which indicates the source [phage] of the gene in question)
    # 3) Keywords (Gene annotation -- doesnt actually have to be anything, but can be used for cool analyses!!)
# We generate this file using the R script: vcontact2_preprocessing.R
    # I did this in R because I am more comfortable with text editing there than in bash

#!/bin/bash
#SBATCH --job-name=vcontact_%j
#SBATCH --nodes=1
#SBATCH -t 12:00:00
#SBATCH --ntasks=16
#SBATCH --output=/home/dsbard/bee_phage/reviewer_approach/network/log/vonctact%j.out
#SBATCH --partition=med

source ~/miniconda3/bin/activate 
conda activate /home/dsbard/yes/envs/vcontact

vcontact2 \
--raw-proteins /home/dsbard/bee_phage/reviewer_approach/network/allseqs.faa \
--proteins-fp /home/dsbard/bee_phage/reviewer_approach/network/gene2genome.tsv \
--output-dir /home/dsbard/bee_phage/reviewer_approach/network/no_ref \
--db None \ # switch to -db 'ProkaryoticViralRefSeq211-Merged' if you want network with reference phage
--rel-mode 'Diamond' \
--pcs-mode MCL \
--vcs-mode ClusterONE \
--c1-bin /home/dsbard/bee_phage/reviewer_approach/scripts/cluster_one-1.0.jar \
--vc-penalty 1 \
-t 16

# This gives a collection of files, some of which are more usefull than others
# The authors provide a great explination of each component here: 
    ## https://bitbucket.org/MAVERICLab/vcontact2/src/master/
# To make pretty networks from these outputs, see the R script titled: decorate_network_all_phage.R